In [ ]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

In [ ]:
df_jan = pd.read_parquet('yellow_tripdata_2023-01.parquet')
df_jan.head()

In [ ]:
# Q1 - How many columns are in the January data? 
print(f"There are {len(df_jan.columns)} columns.") 

In [ ]:
# Q2 What's the standard deviation of the trips duration in January?
df_jan["duration"] = (df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime)
df_jan["duration"] = df_jan["duration"].apply(lambda td: td.total_seconds() / 60)
print(f"The standard deviation of trips in January is {df_jan["duration"].std():.2f} minutes.")


In [ ]:
# Q3 What's the fraction of records left after dropping outliers?
df_jan_out = df_jan[(df_jan["duration"] >= 1) & (df_jan["duration"] <= 60 )]
print(f"After removing outliers we are left with {len(df_jan_out) / len(df_jan)*100:.0f}% of the records.")

In [ ]:
# Q4 - OneHot Encoding
dv = DictVectorizer()
categorical = ["PULocationID", "DOLocationID"]

df_jan_out.loc[:, categorical] = df_jan_out[categorical].astype(str)

train_dicts = df_jan_out[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

print(f"The dimensionality of the training matrix is {X_train.shape[1]}")

In [ ]:
# Q5 - Training the model. RMSE on train
model = LinearRegression()
y_train = df_jan_out["duration"].values
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
rmse_train = root_mean_squared_error(y_train, y_pred)
print(f"The RMSE on train data is: {rmse_train:.2f}")

In [ ]:
# Q6 - Evaluating the model. RMSE on valiudtion
df_feb = pd.read_parquet('yellow_tripdata_2023-02.parquet')
df_feb["duration"] = (df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime)
df_feb["duration"] = df_feb["duration"].apply(lambda td: td.total_seconds() / 60)
df_feb_out = df_feb[(df_feb["duration"] >= 1) & (df_feb["duration"] <= 60 )]

df_feb_out.loc[:, categorical] = df_feb_out[categorical].astype(str)

valid_dicts = df_feb_out[categorical].to_dict(orient='records')
X_valid = dv.transform(valid_dicts)
